In [2]:
import numpy as np
from graph_tool.all import Graph, graph_draw, shortest_path
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo  # 你的自定义模块

# === Step 1: 读取图和节点位置 ===
filename = "./MST_net/tokyo_pop_500_mst.net"
read_graph, read_pos = fo.read_files(filename)

# === Step 2: 计算质心并确定“中心节点” ===
positions = np.array([read_pos[v] for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)
center_node = np.argmin([np.linalg.norm(p - centroid) for p in positions])

# === Step 3: 获取所有叶节点（度为1）===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 4: 将叶节点按到中心的距离进行分层 ===
def assign_layers(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(pos[v] - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: x[1])
    layers = defaultdict(list)
    for idx, (node, dist) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 5: 在每层内添加环形连接，并选定比例连接中心节点 ===
def add_layer_connections(base_graph, pos, layers, center_node, connect_ratio):
    new_edges = []
    for layer, nodes in layers.items():
        # 按极角排序连接成环
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - pos[center_node][1], pos[v][0] - pos[center_node][0]))
        for i in range(len(sorted_nodes)):
            u, v = sorted_nodes[i], sorted_nodes[(i + 1) % len(sorted_nodes)]
            new_edges.append((u, v))
        # 与中心节点连接
        num_to_connect = max(1, int(connect_ratio * len(nodes)))
        chosen = np.random.choice(nodes, num_to_connect, replace=False)
        for node in chosen:
            new_edges.append((center_node, node))
    return new_edges

# === Step 6: 评估总路径和拥堵代价（边负载）===
def evaluate_congestion(graph, edge_list):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())  # 线性拥堵代价
    return total_path_length, congestion_cost

# === Step 7: 设置实验参数（层数 + 中心连接比例）===
layer_options = [2, 3, 4]
connect_ratios = [0.2, 0.5, 0.8]
results = []

# 输出目录
os.makedirs("output", exist_ok=True)

# === Step 8: 实验主循环 ===
for num_layers in layer_options:
    layers = assign_layers(read_pos, leaf_nodes, centroid, num_layers)
    for ratio in connect_ratios:
        new_g = Graph(read_graph)
        added_edges = add_layer_connections(new_g, read_pos, layers, center_node, ratio)
        for u, v in added_edges:
            new_g.add_edge(new_g.vertex(u), new_g.vertex(v))
        total_len, congestion = evaluate_congestion(new_g, added_edges)
        results.append((num_layers, ratio, total_len, congestion))
        # 可视化图保存
        save_path = f"./output/tokyo_layer{num_layers}_ratio{int(ratio*100)}.png"
        graph_draw(new_g, pos=read_pos, output=save_path)

# === Step 9: 保存评估结果为 CSV 表格 ===
df = pd.DataFrame(results, columns=["层数", "中心连接比例", "最短路径总和", "拥堵代价"])
df.to_csv("./output/tokyo_results.csv", index=False)
print("✅ 实验完成！图像已保存至 output 文件夹，结果表格为 tokyo_results.csv")


✅ 实验完成！图像已保存至 output 文件夹，结果表格为 tokyo_results.csv


In [3]:
import numpy as np
from graph_tool.all import Graph, graph_draw, shortest_path
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo  # 自定义模块

# === Step 1: 读取图和节点位置 ===
filename = "./MST_net/tokyo_pop_500_mst.net"
read_graph, read_pos = fo.read_files(filename)

# === Step 2: 计算质心并确定“中心节点” ===
positions = np.array([read_pos[v] for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)
center_node = np.argmin([np.linalg.norm(p - centroid) for p in positions])

# === Step 3: 获取叶节点（度为1）===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 4: 将叶节点按到中心的距离进行分层 ===
def assign_layers(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(pos[v] - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: x[1])
    layers = defaultdict(list)
    for idx, (node, dist) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 5: 每层内构造环状连接（不连中心）===
def add_ring_edges(pos, layers):
    ring_edges = []
    for layer, nodes in layers.items():
        if len(nodes) <= 1:
            continue
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - centroid[1], pos[v][0] - centroid[0]))
        for i in range(len(sorted_nodes)):
            u = sorted_nodes[i]
            v = sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
    return ring_edges

# === Step 6: 可视化绘图函数（区分颜色）===
def draw_graph_colored(base_graph, pos, added_edges, leaf_nodes, save_path):
    v_color = base_graph.new_vertex_property("vector<double>")
    for v in base_graph.vertices():
        if int(v) in leaf_nodes:
            v_color[v] = [1.0, 0.0, 0.0, 1.0]  # 红色
        else:
            v_color[v] = [0.5, 0.5, 0.5, 1.0]  # 灰色

    e_color = base_graph.new_edge_property("vector<double>")
    for e in base_graph.edges():
        s, t = int(e.source()), int(e.target())
        if (s, t) in added_edges or (t, s) in added_edges:
            e_color[e] = [0.0, 0.8, 0.0, 1.0]  # 新增环边绿色
        else:
            e_color[e] = [0.6, 0.6, 0.6, 0.6]  # MST边灰色

    graph_draw(base_graph, pos=pos, vertex_fill_color=v_color, edge_color=e_color,
               vertex_size=5, output=save_path)

# === Step 7: 评估函数（最短路径总和 + 拥堵代价）===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 8: 实验参数设置并执行 ===
layer_options = [2, 3, 4]
results = []

os.makedirs("output", exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges = add_ring_edges(read_pos, layers)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))
    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    # 保存图
    save_path = f"./output/tokyo_mst_plus_ring_L{num_layers}.png"
    draw_graph_colored(new_g, read_pos, ring_edges, leaf_nodes, save_path)

# === Step 9: 保存结果表格 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv("./output/tokyo_ring_only_results.csv", index=False)
print("✅ 已完成图结构构造与保存，评估结果输出为 tokyo_ring_only_results.csv")


✅ 已完成图结构构造与保存，评估结果输出为 tokyo_ring_only_results.csv


In [5]:
import numpy as np
from graph_tool.all import Graph, graph_draw, shortest_path
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo  # 你的自定义模块

# === Step 1: 读取图和节点位置 ===
filename = "./MST_net/tokyo_pop_500_mst.net"
read_graph, read_pos = fo.read_files(filename)

# === Step 2: 计算质心并确定“中心节点” ===
positions = np.array([read_pos[v] for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)
center_node = np.argmin([np.linalg.norm(p - centroid) for p in positions])

# === Step 3: 获取叶节点（度为1）===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 4: 将叶节点按到中心的距离进行分层 ===
def assign_layers(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(pos[v] - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: x[1])
    layers = defaultdict(list)
    for idx, (node, dist) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 5: 最近邻算法构造闭环路径（代替极角排序）===
def tsp_nearest_neighbor(points, pos):
    if len(points) <= 2:
        return points
    visited = [points[0]]
    unvisited = set(points[1:])
    while unvisited:
        last = visited[-1]
        next_node = min(unvisited, key=lambda x: np.linalg.norm(np.array(pos[last]) - np.array(pos[x])))
        visited.append(next_node)
        unvisited.remove(next_node)
    return visited

# === Step 6: 每层内构造环状连接（使用最近邻顺序）===
def add_ring_edges(pos, layers):
    ring_edges = []
    for layer, nodes in layers.items():
        if len(nodes) <= 1:
            continue
        sorted_nodes = tsp_nearest_neighbor(nodes, pos)
        for i in range(len(sorted_nodes)):
            u = sorted_nodes[i]
            v = sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
    return ring_edges

# === Step 7: 可视化图（颜色区分）===
def draw_graph_colored(base_graph, pos, added_edges, leaf_nodes, save_path):
    v_color = base_graph.new_vertex_property("vector<double>")
    for v in base_graph.vertices():
        if int(v) in leaf_nodes:
            v_color[v] = [1.0, 0.0, 0.0, 1.0]  # 红色
        else:
            v_color[v] = [0.5, 0.5, 0.5, 1.0]  # 灰色

    e_color = base_graph.new_edge_property("vector<double>")
    for e in base_graph.edges():
        s, t = int(e.source()), int(e.target())
        if (s, t) in added_edges or (t, s) in added_edges:
            e_color[e] = [0.0, 0.8, 0.0, 1.0]  # 绿色
        else:
            e_color[e] = [0.6, 0.6, 0.6, 0.6]  # 灰色

    graph_draw(base_graph, pos=pos, vertex_fill_color=v_color, edge_color=e_color,
               vertex_size=5, output=save_path)

# === Step 8: 网络评估 ===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 9: 实验参数并执行 ===
layer_options = [2, 3, 4]
results = []

os.makedirs("output", exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges = add_ring_edges(read_pos, layers)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))
    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    # 保存可视化图
    save_path = f"./output/tokyo_mst_plus_ring_L{num_layers}_smooth.png"
    draw_graph_colored(new_g, read_pos, ring_edges, leaf_nodes, save_path)

# === Step 10: 保存评估结果表格 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv("./output/tokyo_ring_tsp_results.csv", index=False)
print("✅ 平滑环状连接构造完成，图像与表格已输出至 ./output 文件夹")


✅ 平滑环状连接构造完成，图像与表格已输出至 ./output 文件夹


In [14]:
import numpy as np
from graph_tool.all import Graph, graph_draw, shortest_path
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from PIL import Image
from matplotlib.patches import Circle
import os
import python_codes.files_operators as fo  # 你的 .net 文件读取模块

# === Step 1: 读取图和节点位置 ===
filename = "./MST_net/tokyo_pop_500_mst.net"
read_graph, read_pos = fo.read_files(filename)

# === Step 2: 计算质心坐标（作为中心点）===
positions = np.array([np.array(read_pos[v]) for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)

# === Step 3: 获取叶节点 ===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 4: 将叶节点按质心距离从远到近分层（外圈编号小）===
def assign_layers(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(np.array(pos[v]) - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: -x[1])
    layers = defaultdict(list)
    for idx, (node, _) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 5: 每层节点按极角排序，构成闭合环 ===
def construct_rings_by_angle(pos, layers, center):
    ring_edges = []
    edge_layer_map = {}
    for layer, nodes in layers.items():
        if len(nodes) <= 2:
            continue
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - center[1], pos[v][0] - center[0]))
        for i in range(len(sorted_nodes)):
            u, v = sorted_nodes[i], sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
            edge_layer_map[(u, v)] = layer
    return ring_edges, edge_layer_map

# === Step 6: 层级颜色设置 ===
def get_layer_colors():
    return {
        0: [1.0, 0.0, 0.0, 1.0],   # 红
        1: [1.0, 0.5, 0.0, 1.0],   # 橙
        2: [1.0, 1.0, 0.0, 1.0],   # 黄
        3: [0.0, 0.8, 0.4, 1.0],   # 绿
        4: [0.0, 0.5, 1.0, 1.0],   # 蓝
        5: [0.6, 0.0, 1.0, 1.0],   # 紫
    }

# === Step 7: 绘图函数（含中心点和辅助圆）===
def draw_graph_colored(base_graph, pos, added_edges, edge_layer_map, leaf_nodes, save_path, centroid=None, layers=None):
    v_color = base_graph.new_vertex_property("vector<double>")
    for v in base_graph.vertices():
        v_color[v] = [1.0, 0.0, 0.0, 1.0] if int(v) in leaf_nodes else [0.5, 0.5, 0.5, 1.0]

    layer_colors = get_layer_colors()
    e_color = base_graph.new_edge_property("vector<double>")
    for e in base_graph.edges():
        s, t = int(e.source()), int(e.target())
        edge = (s, t) if (s, t) in edge_layer_map else (t, s)
        e_color[e] = layer_colors.get(edge_layer_map[edge], [0.6, 0.6, 0.6, 0.6]) if edge in edge_layer_map else [0.6, 0.6, 0.6, 0.6]

    # Step 1: 输出基础图像
    temp_path = save_path.replace(".png", "_base.png")
    graph_draw(base_graph, pos=pos,
               vertex_fill_color=v_color,
               edge_color=e_color,
               vertex_size=5,
               output=temp_path)

    # Step 2: 叠加辅助元素（中心点 + 圆环）
    img = Image.open(temp_path)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img)
    ax.axis('off')

    if centroid is not None:
        ax.plot(centroid[0], centroid[1], marker='x', color='lime', markersize=10)

    if layers is not None:
        for layer, nodes in layers.items():
            if not nodes:
                continue
            dists = [np.linalg.norm(np.array(pos[v]) - centroid) for v in nodes]
            avg_radius = np.mean(dists)
            circle = Circle(centroid, avg_radius, edgecolor='lightgreen',
                            linestyle='--', linewidth=1.2, fill=False)
            ax.add_patch(circle)

    plt.savefig(save_path, dpi=300)
    plt.close()

# === Step 8: 网络路径和拥堵代价评估 ===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 9: 主流程（多层结构绘制与评估）===
layer_options = [2, 3, 4]
results = []

os.makedirs("output", exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges, edge_layer_map = construct_rings_by_angle(read_pos, layers, centroid)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))
    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    save_path = f"./output/tokyo_mst_ring_L{num_layers}.png"
    draw_graph_colored(new_g, read_pos, ring_edges, edge_layer_map, leaf_nodes,
                       save_path, centroid=centroid, layers=layers)

# === Step 10: 输出 CSV 表格结果 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv("./output/tokyo_ring_results.csv", index=False)

print("✅ 所有图与数据已保存至 ./output/")


✅ 所有图与数据已保存至 ./output/


In [19]:
import numpy as np
from graph_tool.all import Graph, shortest_path
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo

# === Step 1: 读取图与节点位置 ===
filename = "./MST_net/tokyo_pop_500_mst.net"
read_graph, read_pos = fo.read_files(filename)

# === Step 2: 计算质心 ===
positions = np.array([np.array(read_pos[v]) for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)

# === Step 3: 获取叶节点 ===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 4: 分层函数 ===
def assign_layers(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(np.array(pos[v]) - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: -x[1])
    layers = defaultdict(list)
    for idx, (node, _) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 5: 构建每层的环连接（按角度排序）===
def construct_rings_by_angle(pos, layers, center):
    ring_edges = []
    edge_layer_map = {}
    for layer, nodes in layers.items():
        if len(nodes) <= 2:
            continue
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - center[1], pos[v][0] - center[0]))
        for i in range(len(sorted_nodes)):
            u, v = sorted_nodes[i], sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
            edge_layer_map[(u, v)] = layer
    return ring_edges, edge_layer_map

# === Step 6: 层级颜色 ===
def get_layer_colors():
    return {
        0: 'red',
        1: 'orange',
        2: 'yellow',
        3: 'green',
        4: 'blue',
        5: 'purple'
    }

# === Step 7: matplotlib绘图函数 ===
def draw_graph_matplotlib(g, pos, added_edges, edge_layer_map, leaf_nodes, save_path, centroid=None, layers=None):
    fig, ax = plt.subplots(figsize=(8, 8))

    # 原始边（MST）灰色绘制
    for e in g.edges():
        s, t = int(e.source()), int(e.target())
        x = [pos[s][0], pos[t][0]]
        y = [pos[s][1], pos[t][1]]
        ax.plot(x, y, color='gray', linewidth=0.6, zorder=1)

    # 新加边（环）按层颜色绘制
    layer_colors = get_layer_colors()
    for (u, v) in added_edges:
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        layer = edge_layer_map.get((u, v), edge_layer_map.get((v, u), 0))
        ax.plot(x, y, color=layer_colors.get(layer, 'red'), linewidth=1.0, zorder=2)

    # 节点绘制
    for v in g.vertices():
        x, y = pos[int(v)][0], pos[int(v)][1]
        if int(v) in leaf_nodes:
            ax.scatter(x, y, color='red', s=8, zorder=3)
        else:
            ax.scatter(x, y, color='gray', s=8, zorder=2)

    # 绘制中心点（绿色 X）
    if centroid is not None:
        ax.scatter(centroid[0], centroid[1], color='lime', marker='x', s=60, zorder=4)

    # 绘制每一层的平均辅助圆
    if layers is not None:
        for layer, nodes in layers.items():
            if not nodes:
                continue
            dists = [np.linalg.norm(np.array(pos[v]) - centroid) for v in nodes]
            avg_radius = np.mean(dists)
            circle = Circle(centroid, avg_radius, edgecolor='lightgreen',
                            linestyle='--', linewidth=1.0, fill=False, zorder=0)
            ax.add_patch(circle)

    ax.set_aspect('equal')
    ax.axis('off')
    ax.invert_yaxis()  
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

# === Step 8: 评估函数（路径长度和拥堵）===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 9: 主流程（遍历不同层数）===
layer_options = [2, 3, 4, 5, 6, 7]
results = []

os.makedirs("output", exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges, edge_layer_map = construct_rings_by_angle(read_pos, layers, centroid)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))
    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    save_path = f"./output/tokyo_mst_ring_L{num_layers}.png"
    draw_graph_matplotlib(new_g, read_pos, ring_edges, edge_layer_map, leaf_nodes,
                          save_path, centroid=centroid, layers=layers)

# === Step 10: 输出表格 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv("./output/tokyo_ring_results.csv", index=False)

print("✅ 所有图与数据已保存至 ./output/")


✅ 所有图与数据已保存至 ./output/


In [20]:
import numpy as np
from graph_tool.all import Graph, shortest_path
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo

# === Step 1: 读取图与节点位置 ===
filename = "./MST_net/2keihan2_pop_500_mst.net"
read_graph, read_pos = fo.read_files(filename)

# === Step 2: 计算质心 ===
positions = np.array([np.array(read_pos[v]) for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)

# === Step 3: 获取叶节点 ===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 4: 分层函数 ===
def assign_layers(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(np.array(pos[v]) - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: -x[1])
    layers = defaultdict(list)
    for idx, (node, _) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 5: 构建每层的环连接（按角度排序）===
def construct_rings_by_angle(pos, layers, center):
    ring_edges = []
    edge_layer_map = {}
    for layer, nodes in layers.items():
        if len(nodes) <= 2:
            continue
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - center[1], pos[v][0] - center[0]))
        for i in range(len(sorted_nodes)):
            u, v = sorted_nodes[i], sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
            edge_layer_map[(u, v)] = layer
    return ring_edges, edge_layer_map

# === Step 6: 层级颜色 ===
def get_layer_colors():
    return {
        0: 'red',
        1: 'orange',
        2: 'yellow',
        3: 'green',
        4: 'blue',
        5: 'purple'
    }

# === Step 7: matplotlib绘图函数 ===
def draw_graph_matplotlib(g, pos, added_edges, edge_layer_map, leaf_nodes, save_path, centroid=None, layers=None):
    fig, ax = plt.subplots(figsize=(8, 8))

    # 原始边（MST）灰色绘制
    for e in g.edges():
        s, t = int(e.source()), int(e.target())
        x = [pos[s][0], pos[t][0]]
        y = [pos[s][1], pos[t][1]]
        ax.plot(x, y, color='gray', linewidth=0.6, zorder=1)

    # 新加边（环）按层颜色绘制
    layer_colors = get_layer_colors()
    for (u, v) in added_edges:
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        layer = edge_layer_map.get((u, v), edge_layer_map.get((v, u), 0))
        ax.plot(x, y, color=layer_colors.get(layer, 'red'), linewidth=1.0, zorder=2)

    # 节点绘制
    for v in g.vertices():
        x, y = pos[int(v)][0], pos[int(v)][1]
        if int(v) in leaf_nodes:
            ax.scatter(x, y, color='red', s=8, zorder=3)
        else:
            ax.scatter(x, y, color='gray', s=8, zorder=2)

    # 绘制中心点（绿色 X）
    if centroid is not None:
        ax.scatter(centroid[0], centroid[1], color='lime', marker='x', s=60, zorder=4)

    # 绘制每一层的平均辅助圆
    if layers is not None:
        for layer, nodes in layers.items():
            if not nodes:
                continue
            dists = [np.linalg.norm(np.array(pos[v]) - centroid) for v in nodes]
            avg_radius = np.mean(dists)
            circle = Circle(centroid, avg_radius, edgecolor='lightgreen',
                            linestyle='--', linewidth=1.0, fill=False, zorder=0)
            ax.add_patch(circle)

    ax.set_aspect('equal')
    ax.axis('off')
    ax.invert_yaxis()  
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

# === Step 8: 评估函数（路径长度和拥堵）===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 9: 主流程（遍历不同层数）===
layer_options = [2, 3, 4, 5, 6, 7]
results = []

os.makedirs("output", exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges, edge_layer_map = construct_rings_by_angle(read_pos, layers, centroid)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))
    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    save_path = f"./output/keihan_mst_ring_L{num_layers}.png"
    draw_graph_matplotlib(new_g, read_pos, ring_edges, edge_layer_map, leaf_nodes,
                          save_path, centroid=centroid, layers=layers)

# === Step 10: 输出表格 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv("./output/keihan_ring_results.csv", index=False)

print("✅ 所有图与数据已保存至 ./output/")


✅ 所有图与数据已保存至 ./output/


In [23]:
import numpy as np
from graph_tool.all import Graph, shortest_path
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo

# === Step 1: 设置城市与规模 ===
city = "tokyo"
scale = "1000"
filename = f"./MST_net/{city}_pop_{scale}_mst.net"

# === Step 2: 读取图与节点位置 ===
read_graph, read_pos = fo.read_files(filename)

# === Step 3: 计算质心 ===
positions = np.array([np.array(read_pos[v]) for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)

# === Step 4: 获取叶节点 ===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 5: 节点数量平均分层 ===
def assign_layers_equal_count(pos, leaf_nodes, center, num_layers):
    distances = {v: np.linalg.norm(np.array(pos[v]) - center) for v in leaf_nodes}
    sorted_nodes = sorted(distances.items(), key=lambda x: -x[1])  # 从远到近
    layers = defaultdict(list)
    for idx, (node, _) in enumerate(sorted_nodes):
        layer = int((idx / len(leaf_nodes)) * num_layers)
        layers[layer].append(node)
    return layers

# === Step 6: 每层构建环（极角排序）===
def construct_rings_by_angle(pos, layers, center):
    ring_edges = []
    edge_layer_map = {}
    for layer, nodes in layers.items():
        if len(nodes) <= 2:
            continue
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - center[1], pos[v][0] - center[0]))
        for i in range(len(sorted_nodes)):
            u, v = sorted_nodes[i], sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
            edge_layer_map[(u, v)] = layer
    return ring_edges, edge_layer_map

# === Step 7: 层级颜色 ===
def get_layer_colors():
    return {
        0: 'red',
        1: 'orange',
        2: 'yellow',
        3: 'green',
        4: 'blue',
        5: 'purple',
        6: 'brown'
    }

# === Step 8: 绘图函数（带叶节点数量标注）===
def draw_graph_with_counts(g, pos, added_edges, edge_layer_map, leaf_nodes, save_path, centroid=None, layers=None):
    fig, ax = plt.subplots(figsize=(8, 8))

    # 原始 MST 边
    for e in g.edges():
        s, t = int(e.source()), int(e.target())
        x = [pos[s][0], pos[t][0]]
        y = [pos[s][1], pos[t][1]]
        ax.plot(x, y, color='gray', linewidth=0.6, zorder=1)

    # 新连接边
    layer_colors = get_layer_colors()
    for (u, v) in added_edges:
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        layer = edge_layer_map.get((u, v), edge_layer_map.get((v, u), 0))
        ax.plot(x, y, color=layer_colors.get(layer, 'red'), linewidth=1.0, zorder=2)

    # 节点绘制
    for v in g.vertices():
        x, y = pos[int(v)][0], pos[int(v)][1]
        if int(v) in leaf_nodes:
            ax.scatter(x, y, color='red', s=8, zorder=3)
        else:
            ax.scatter(x, y, color='gray', s=8, zorder=2)

    # 中心点
    if centroid is not None:
        ax.scatter(centroid[0], centroid[1], color='lime', marker='x', s=60, zorder=4)

    # 辅助圆 & 数量标注
    if layers is not None:
        for layer, nodes in layers.items():
            if not nodes:
                continue
            dists = [np.linalg.norm(np.array(pos[v]) - centroid) for v in nodes]
            avg_radius = np.mean(dists)
            circle = Circle(centroid, avg_radius, edgecolor='lightgreen',
                            linestyle='--', linewidth=1.0, fill=False, zorder=0)
            ax.add_patch(circle)

            # 添加文本：节点数量标注
            angle = np.pi / 4
            label_x = centroid[0] + avg_radius * np.cos(angle)
            label_y = centroid[1] + avg_radius * np.sin(angle)
            ax.text(label_x, label_y, f"{len(nodes)} nodes", fontsize=8,
                    color=layer_colors.get(layer, 'black'), ha='center')

    ax.set_aspect('equal')
    ax.axis('off')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

# === Step 9: 拓扑评估 ===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 10: 主流程 ===
layer_options = list(range(2, 8))  # 2 到 7 层
results = []

output_dir = f"./output/rings_by_equal_count/{city}_pop_{scale}_mst"
os.makedirs(output_dir, exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers_equal_count(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges, edge_layer_map = construct_rings_by_angle(read_pos, layers, centroid)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))

    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    save_path = f"{output_dir}/{city}_mst_ring_L{num_layers}.png"
    draw_graph_with_counts(new_g, read_pos, ring_edges, edge_layer_map, leaf_nodes,
                           save_path, centroid=centroid, layers=layers)

# === Step 11: 输出表格 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv(f"{output_dir}/{city}_ring_results.csv", index=False)

print(f"✅ 完成！图与数据已保存至 {output_dir}/")


✅ 完成！图与数据已保存至 ./output/rings_by_equal_count/tokyo_pop_1000_mst/
